# Next Word Prediction

## Introduction and Objective

In the era of advanced natural language processing, predictive text systems have become an integral part of user interfaces, enhancing user experience by providing suggestions and auto-completing text inputs. This notebook demonstrates the implementation of a next word prediction system using an N-gram model with Good-Turing smoothing and interpolation techniques. An N-gram model is a type of probabilistic language model that predicts the next item in a sequence based on the previous items, making it a powerful tool for understanding and generating human language.

The primary objective of this project is to build a robust word prediction system by leveraging statistical language modeling techniques. The N-gram model is trained on a large corpus of text data, allowing it to learn the probabilities of sequences of words. To address the issue of zero probabilities for unseen N-grams, Good-Turing smoothing is applied, which adjusts the probability estimates for rare or unseen events. Additionally, interpolation combines the probabilities of different N-grams (e.g., bigrams, trigrams) to improve prediction accuracy. This combination of techniques ensures that the model can make accurate and reliable predictions, even when encountering new or uncommon word sequences.

## Dataset

The dataset used for training and evaluating the N-gram model is included in this repository. Specifically, the following files are used:
- `internet_archive_scifi_v3.txt`: This file is used for training the N-gram model. It contains a large corpus of science fiction text data from which the model learns the probabilities of word sequences.

You can find the dataset at this [Kaggle link](https://www.kaggle.com/datasets/jannesklaas/scifi-stories-text-corpus).

This science fiction dataset will be utilized to complete sentences and generate text relevant to the genre, ensuring the model can capture the unique vocabulary and stylistic elements typical of science fiction narratives.

## Installing Libraries

To enhance the functionality of the environment, you may need to install some libraries not pre-installed in the CTPO environment but required for this notebook. Follow these steps to install the necessary libraries from the `requirements.txt` file:

### 1. Create and Activate the Virtual Environment

Open your terminal or command prompt within the Jupyter notebook. Navigate to `File -> New -> Terminal` and type `bash` to get a shell compatible with the following commands.

Navigate to the project directory where the notebook is to set up the environment.

Execute the following commands to create and activate the virtual environment:

```sh
bash
python3 -m venv --system-site-packages myvenv
source myvenv/bin/activate
pip3 install ipykernel
python -m ipykernel install --user --name=myvenv --display-name="Python (myvenv)"
```

### 2. Install Required Libraries

Before running the following command in the Jupyter notebook, make sure you are in the directory where the Jupyter Notebook and virtual environment is located. Load the newly created "Python (myvenv)" kernel. This ensures the `./` path is always current. You can use the `cd` command to change to your project directory and `pwd` to verify your current directory.

#### Important Note
It is crucial to load the new "myvenv" kernel for the notebook to work correctly. If the new "myvenv" kernel is not loaded, the required libraries and environment settings will not be applied, and the notebook will not function as expected.



In [1]:
!. ./myvenv/bin/activate; pip install -r requirements.txt

In [2]:
from nltk.util import ngrams
from collections import defaultdict
from collections import OrderedDict
import string
import time
import gc
from math import log10
start_time = time.time()

The function `removePunctuations(sen)` processes an input string by splitting it into words, removing punctuation from each word (except possessive apostrophes), converting the words to lowercase, and then joining the words back into a single string. This cleaned string is then returned.

In [3]:
def removePunctuations(sen):
    #split the string into word tokens
    temp_l = sen.split()
    #print(temp_l)
    i = 0
    j = 0
    
    #changes the word to lowercase and removes punctuations from it
    for word in temp_l :
        j = 0
        #print(len(word))
        for l in word :
            if l in string.punctuation:
                if l == "'":
                    if j+1<len(word) and word[j+1] == 's':
                        j = j + 1
                        continue
                word = word.replace(l," ")
                #print(j,word[j])
            j += 1

        temp_l[i] = word.lower()
        i=i+1   

    #spliting is being don here beacause in sentences line here---so after punctuation removal it should 
    #become "here so"   
    content = " ".join(temp_l)

    return content

The function `loadCorpus(file_path, bi_dict, tri_dict, quad_dict, vocab_dict)` reads a corpus file line by line, processes each line to remove punctuation and convert words to lowercase, and then generates bigrams, trigrams, and quadgrams. It updates the dictionaries with the frequency counts of these n-grams and the vocabulary. It also ensures continuity across lines by storing the last three words of each line for pairing with the next line.

In [4]:
def loadCorpus(file_path, bi_dict, tri_dict, quad_dict, vocab_dict):

    w1 = ''    #for storing the 3rd last word to be used for next token set
    w2 = ''    #for storing the 2nd last word to be used for next token set
    w3 = ''    #for storing the last word to be used for next token set
    token = []
    #total no. of words in the corpus
    word_len = 0

    #open the corpus file and read it line by line
    with open(file_path,'r') as file:
        for line in file:

            #split the string into word tokens
            temp_l = line.split()
            i = 0
            j = 0
            
            #does the same as the removePunctuations() function,implicit declratation for performance reasons
            #changes the word to lowercase and removes punctuations from it
            for word in temp_l :
                j = 0
                #print(len(word))
                for l in word :
                    if l in string.punctuation:
                        if l == "'":
                            if j+1<len(word) and word[j+1] == 's':
                                j = j + 1
                                continue
                        word = word.replace(l," ")
                        #print(j,word[j])
                    j += 1

                temp_l[i] = word.lower()
                i=i+1   

            #spliting is being done here beacause in sentences line here---so after punctuation removal it should 
            #become "here so"   
            content = " ".join(temp_l)

            token = content.split()
            word_len = word_len + len(token)  

            if not token:
                continue

            #add the last word from previous line
            if w3!= '':
                token.insert(0,w3)

            temp0 = list(ngrams(token,2))

            #since we are reading line by line some combinations of word might get missed for pairing
            #for trigram
            #first add the previous words
            if w2!= '':
                token.insert(0,w2)

            #tokens for trigrams
            temp1 = list(ngrams(token,3))

            #insert the 3rd last word from previous line for quadgram pairing
            if w1!= '':
                token.insert(0,w1)

            #add new unique words to the vocaulary set if available
            for word in token:
                if word not in vocab_dict:
                    vocab_dict[word] = 1
                else:
                    vocab_dict[word]+= 1
                  
            #tokens for quadgrams
            temp2 = list(ngrams(token,4))

            #count the frequency of the bigram sentences
            for t in temp0:
                sen = ' '.join(t)
                bi_dict[sen] += 1

            #count the frequency of the trigram sentences
            for t in temp1:
                sen = ' '.join(t)
                tri_dict[sen] += 1

            #count the frequency of the quadgram sentences
            for t in temp2:
                sen = ' '.join(t)
                quad_dict[sen] += 1


            #then take out the last 3 words
            n = len(token)

            #store the last few words for the next sentence pairing
            if (n -3) >= 0:
                w1 = token[n -3]
            if (n -2) >= 0:
                w2 = token[n -2]
            if (n -1) >= 0:
                w3 = token[n -1]
    return word_len

The function `findQuadgramProbGT(vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, nc_dict, k)` calculates the probabilities of quadgrams using Good-Turing smoothing. For each quadgram, it splits the quadgram into tokens and derives the corresponding trigram. It then applies Good-Turing smoothing to adjust the counts of both the quadgram and trigram if their counts are less than or equal to `k`. The probability of each quadgram is calculated as the ratio of the adjusted quadgram count to the adjusted trigram count. These probabilities are stored in `quad_prob_dict`, with the trigrams as keys and the list of quadgram probabilities and their last word as values.

In [5]:
def findQuadgramProbGT(vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, nc_dict, k):
    
    i = 0
    V = len(vocab_dict)
   
    for quad_sen in quad_dict:
        quad_token = quad_sen.split()
        
        #trigram sentence for key
        tri_sen = ' '.join(quad_token[:3])

        #find the probability
        #Good Turing smoothing has been used
        quad_count = quad_dict[quad_sen]
        tri_count = tri_dict[tri_sen]
        
        if quad_dict[quad_sen] <= k  or (quad_sen not in quad_dict):
            quad_count = findGoodTuringAdjustCount( quad_dict[quad_sen], k, nc_dict)
        if tri_dict[tri_sen] <= k  or (tri_sen not in tri_dict):
            tri_count = findGoodTuringAdjustCount( tri_dict[tri_sen], k, nc_dict)
        
        prob = quad_count / tri_count
        
        #add the trigram to the quadgram probabiltity dict
        if tri_sen not in quad_prob_dict:
            quad_prob_dict[tri_sen] = []
            quad_prob_dict[tri_sen].append([prob,quad_token[-1]])
        else:
            quad_prob_dict[tri_sen].append([prob,quad_token[-1]])
  
    prob = None
    quad_token = None
    tri_sen = None

 This process of `findTrigramProbGT`  is similar to `findQuadgramProbGT`, but operates on trigrams instead of quadgrams.

In [6]:
def findTrigramProbGT(vocab_dict, bi_dict, tri_dict, tri_prob_dict, nc_dict, k):
    
    #vocabulary length
    V = len(vocab_dict)
    
    #create a dictionary of probable words with their probabilities for
    #trigram probabilites,key is a bigram and value is a list of prob and word
    for tri in tri_dict:
        tri_token = tri.split()
        #bigram sentence for key
        bi_sen = ' '.join(tri_token[:2])
        
        #find the probability
        #Good Turing smoothing has been used
        tri_count = tri_dict[tri]
        bi_count = bi_dict[bi_sen]
        
        if tri_dict[tri] <= k or (tri not in tri_dict):
            tri_count = findGoodTuringAdjustCount( tri_dict[tri], k, nc_dict)
        if bi_dict[bi_sen] <= k or (bi_sen not in bi_dict):
            bi_count = findGoodTuringAdjustCount( bi_dict[bi_sen], k, nc_dict)
        
        prob = tri_count / bi_count
        
        #add the bigram sentence  to the trigram probability dict
        #tri_prob_dict is a dict of list
        if bi_sen not in tri_prob_dict:
            tri_prob_dict[bi_sen] = []
            tri_prob_dict[bi_sen].append([prob,tri_token[-1]])
        else:
            tri_prob_dict[bi_sen].append([prob,tri_token[-1]])
    
    prob = None
    tri_token = None
    bi_sen = None

 This process of `findBigramProbGT` is similar to `findQuadgramProbGT`, but operates on bigrams instead of quadgrams.

In [7]:
def findBigramProbGT(vocab_dict, bi_dict, bi_prob_dict, nc_dict, k):
   
    #vocabulary size
    V = len(vocab_dict)
    
    #create a dictionary of probable words with their probabilities for bigram probabilites
    for bi in bi_dict:
        bi_token = bi.split()
        #unigram for key
        unigram = bi_token[0]
       
        #find the probability
        #Good Turing smoothing has been used
        bi_count = bi_dict[bi]
        uni_count = vocab_dict[unigram]
        
        if bi_dict[bi] <= k or (bi not in bi_dict):
            bi_count = findGoodTuringAdjustCount( bi_dict[bi], k, nc_dict)
        if vocab_dict[unigram] <= k or (unigram not in vocab_dict):
            uni_count = findGoodTuringAdjustCount( vocab_dict[unigram], k, nc_dict)
        
        prob = bi_count / uni_count
        
        #add the unigram to the bigram probability dict
        #bi_prob_dict is a dict of list
        if unigram not in bi_prob_dict:
            bi_prob_dict[unigram] = []
            bi_prob_dict[unigram].append([prob,bi_token[-1]])
        else:
            bi_prob_dict[unigram].append([prob,bi_token[-1]])
    
   
    prob = None
    bi_token = None
    unigram = None

The function `sortProbWordDict` sorts the probability lists within the `bi_prob_dict`, `tri_prob_dict`, and `quad_prob_dict` dictionaries in descending order. For `quad_prob_dict`, it keeps only the top two probabilities after sorting.

In [8]:
def sortProbWordDict(bi_prob_dict, tri_prob_dict, quad_prob_dict):
    for key in bi_prob_dict:
        if len(bi_prob_dict[key])>1:
            bi_prob_dict[key] = sorted(bi_prob_dict[key],reverse = True)
    
    for key in tri_prob_dict:
        if len(tri_prob_dict[key])>1:
            tri_prob_dict[key] = sorted(tri_prob_dict[key],reverse = True)
    
    for key in quad_prob_dict:
        if len(quad_prob_dict[key])>1:
            quad_prob_dict[key] = sorted(quad_prob_dict[key],reverse = True)[:2]

The `takeInput` function prompts the user to input a string, removes any punctuation, and ensures that the input contains at least three words. If the input is valid, it returns the last three words as a single string.

In [9]:
def takeInput():
    cond = False
    #take input
    while(cond == False):
        sen = input('Enter the string\n')
        sen = removePunctuations(sen)
        temp = sen.split()
        if len(temp) < 3:
            print("Please enter atleast 3 words !")
        else:
            cond = True
    sen = " ".join(temp)
    return sen

In [10]:
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import style

#finds the slope for the best fit line
def findBestFitSlope(x,y):
    m = (( mean(x)*mean(y) - mean(x*y) ) / 
          ( mean(x)** 2 - mean(x**2)))

    return m
      
#finds the intercept for the best fit line
def findBestFitIntercept(x,y,m):
    c = mean(y) - m*mean(x)
    return c

The function `findFrequencyOfFrequencyCount` calculates the frequency of frequencies (`Nc`) for n-grams in `ngram_dict` up to a count `k`. It ensures all counts up to `k+1` are present, filling any missing values using linear regression based on the log-log relationship between `c` and `Nc`. This helps in applying Good-Turing smoothing by estimating counts for unseen n-grams.

In [11]:
def findFrequencyOfFrequencyCount(ngram_dict, k, n, V, token_len):
    #for keeping count of 'c' value i.e Nc
    nc_dict = {}
    #we find the value of Nc,c = 0 by V^n - (total n-gram tokens)
    nc_dict[0] = V**n - token_len
    #find the count Nc till c = k,we will take k = 5
    #find counts for n-gram
    for key in ngram_dict:
        if ngram_dict[key] <= k + 1:
            if ngram_dict[key] not in nc_dict:
                nc_dict[ ngram_dict[key]] = 1
            else:
                nc_dict[ ngram_dict[key] ] += 1
    
    #check if all the values of Nc are there in the nc_dict or not ,if there then return           
    val_present = True
    for i in range(1,7):
        if i not in nc_dict:
            val_present = False
            break
    if val_present == True:
        return nc_dict
    
    #now fill in the values of nc in case it is not there using regression upto c = 6
    #we use :[ log(Nc) = blog(c) + a ] as the equation

    #we first need to find data for regression that is values(Nc,c) we take 5 data points
    data_pts = {}
    i = 0
    #get first 5 counts value i.e c
    #for quadgram
    for key in ngram_dict:
        if ngram_dict[key] not in data_pts:
                data_pts[ ngram_dict[key] ] = 1
                i += 1
        if i >5:
            break
            
    #now get Nc for those c values
    for key in ngram_dict:
        if ngram_dict[key] in data_pts:
            data_pts[ ngram_dict[key] ] += 1
    
    #make x ,y coordinates for regression 
    x_coor = [ np.log(item) for item in data_pts ]
    y_coor = [ np.log( data_pts[item] ) for item in data_pts ]
    x = np.array(x_coor, dtype = np.float64)
    y = np.array(y_coor , dtype = np.float64)
   

    #now do regression
    #find the slope and intercept for the regression equation
    slope_m = findBestFitSlope(x,y)
    intercept_c = findBestFitIntercept(x,y,slope_m)

    #now find the missing Nc terms and give them value using regression
    for i in range(1,(k+2)):
        if i not in nc_dict:
            nc_dict[i] = (slope_m*i) + intercept_c
    
    return nc_dict

The function `findGoodTuringAdjustCount` calculates the Good-Turing adjusted count for a given frequency `c` using the `nc_dict` that contains frequency-of-frequency counts. It adjusts the count to account for low-frequency events and provides a more accurate probability estimate for rare n-grams.

In [12]:
def findGoodTuringAdjustCount(c, k, nc_dict):
   
    adjust_count = ( ( (( c + 1)*( nc_dict[c + 1] / nc_dict[c])) - ( c * (k+1) * nc_dict[k+1] / nc_dict[1]) ) /
                     ( 1 - (( k + 1)*nc_dict[k + 1] / nc_dict[1]) )
                   )
    return adjust_count

The function `doPredictionBackoffGT` predicts the next word in a sequence using a backoff model. It first checks for the highest-order n-gram (quadgram) in `quad_prob_dict`, and if not found, it backs off to trigrams (`tri_prob_dict`) and then bigrams (`bi_prob_dict`), returning the most probable word if a valid prediction is found.

In [19]:
def doPredictionBackoffGT(input_sen, bi_dict, tri_dict, quad_dict, bi_prob_dict, tri_prob_dict, quad_prob_dict):

    tokens = input_sen.split()
    

    pred = []
    

    if len(tokens) >= 3:
        quad_key = ' '.join(tokens[-3:])
        if quad_key in quad_prob_dict and quad_prob_dict[quad_key][0][0] > 0:
            pred = quad_prob_dict[quad_key][0]
            return pred
    
    # Check trigrams
    if len(tokens) >= 2:
        tri_key = ' '.join(tokens[-2:])
        if tri_key in tri_prob_dict and tri_prob_dict[tri_key][0][0] > 0:
            pred = tri_prob_dict[tri_key][0]
            return pred
    
    # Check bigrams
    if len(tokens) >= 1:
        bi_key = tokens[-1]
        if bi_key in bi_prob_dict and bi_prob_dict[bi_key][0][0] > 0:
            pred = bi_prob_dict[bi_key][0]
            return pred
    
    # Return the prediction (empty if no match found)
    return pred


The `word_prediction` function initializes dictionaries to store n-gram frequencies and probabilities, loads and processes the training corpus, creates n-gram probability dictionaries with Good-Turing smoothing, and sorts these dictionaries. It then takes user input, predicts the next word using the backoff model, and prints the prediction if available.

In [20]:
def dictss(train_file):
    vocab_dict = defaultdict(int)          #for storing the different words with their frequencies    
    bi_dict = defaultdict(int)             #for keeping count of sentences of two words
    tri_dict = defaultdict(int)            #for keeping count of sentences of three words
    quad_dict = defaultdict(int)           #for keeping count of sentences of four words
    quad_prob_dict = OrderedDict()              
    tri_prob_dict = OrderedDict()
    bi_prob_dict = OrderedDict()
    
        #load corpus
    token_len = loadCorpus(train_file, bi_dict, tri_dict, quad_dict, vocab_dict)
    
        #create the different Nc dictionaries for ngrams
        #threshold value
    k = 5
    V = len(vocab_dict)
    quad_nc_dict = findFrequencyOfFrequencyCount(quad_dict, k, 4, V, len(quad_dict))
    tri_nc_dict = findFrequencyOfFrequencyCount(tri_dict, k, 3, V, len(tri_dict))
    bi_nc_dict = findFrequencyOfFrequencyCount(bi_dict, k, 2, V, len(bi_dict))
    uni_nc_dict = findFrequencyOfFrequencyCount(bi_dict, k, 1, V, len(vocab_dict))
    return vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, quad_nc_dict,k, tri_prob_dict,tri_nc_dict,bi_prob_dict,bi_nc_dict

def word_prediction(input_sen,vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, quad_nc_dict,k, tri_prob_dict,tri_nc_dict,bi_prob_dict,bi_nc_dict):
    ##WORD PREDICTION 
    #take user input 
    print('The sentence given by user is: ', input_sen)

    prediction = doPredictionBackoffGT(input_sen, bi_dict, tri_dict, quad_dict, bi_prob_dict, tri_prob_dict, quad_prob_dict)
    if prediction:
        print('Word Prediction:',prediction[1])

In [15]:
# load the corpus
vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, quad_nc_dict,k, tri_prob_dict,tri_nc_dict,bi_prob_dict,bi_nc_dict= dictss('internet_archive_scifi_v3.txt')

In [ ]:
## variable declaration
#create quadgram probability dictionary
findQuadgramProbGT(vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, quad_nc_dict, k)
#create trigram probability dictionary
findTrigramProbGT(vocab_dict, bi_dict, tri_dict, tri_prob_dict, tri_nc_dict, k)
#create bigram probability dictionary
findBigramProbGT(vocab_dict, bi_dict, bi_prob_dict, bi_nc_dict, k)
#sort the probability dictionaries of quad,tri and bi grams
sortProbWordDict(bi_prob_dict, tri_prob_dict, quad_prob_dict)

In [ ]:
input_sen = "rates for IF are set for 6 issues in the U.S. and"
word_prediction(input_sen,vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, quad_nc_dict,k, tri_prob_dict,tri_nc_dict,bi_prob_dict,bi_nc_dict)

In [ ]:
input_sen = "Naia North confesses to being the"
word_prediction(input_sen,vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, quad_nc_dict,k, tri_prob_dict,tri_nc_dict,bi_prob_dict,bi_nc_dict)

In [21]:
input_sen= takeInput()
word_prediction(input_sen,vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict, quad_nc_dict,k, tri_prob_dict,tri_nc_dict,bi_prob_dict,bi_nc_dict)

Enter the string
 Subscription rates for IF are set for 6 issues in the U.S. and 
